In [96]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from pandas import read_excel

from sklearn import metrics

import xgboost as xgb

from sklearn.preprocessing import OneHotEncoder
#from feature_engine import categorical_encoders as ce
from sklearn.pipeline import Pipeline

from sklearn import model_selection

import category_encoders as ce

In [98]:
#my_sheet = 'Sheet1' # change it to your sheet name, you can find your sheet name at the bottom left of your excel file
file_name = '../data/corridas.xlsx' # change it to the name of your excel file
df = read_excel(file_name)

In [99]:
df

,Date,Distancia,Duration,Average Pace,Average Speed (km/h),Calories Burned,Climb (m)
0,2022-02-13 09:53:12,5.01,34:22,6:51,875.0,3220,147
1,2022-02-06 09:34:34,2.90,33:30,11:34,519.0,1800,93
2,2022-02-05 09:14:42,8.00,54:55,6:52,875.0,5260,240
3,2022-02-02 07:39:37,5.09,35:05,6:53,871.0,3370,172
4,2022-01-30 10:58:47,5.42,37:12,6:52,874.0,3550,164
...,...,...,...,...,...,...,...
390,2015-05-31 08:45:46,17.30,1:55:01,6:39,902.0,13810,266
391,2015-05-24 10:20:25,13.62,1:35:20,7:00,857.0,10800,208
392,2015-05-03 09:44:33,10.89,1:20:27,7:23,812.0,8430,187
393,2015-04-26 10:52:26,11.10,1:12:07,6:30,924.0,8290,165


In [100]:
df['Week_Number'] = df['Date'].dt.year.astype(str) + '-' + (df['Date'].dt.week + 100).astype(str).str[1:3]
df['dia_semana'] = (df['Date'].dt.weekday + 100).astype(str).str[1:3]
df

C:\Users\csati\AppData\Local\Temp/ipykernel_11124/1195871656.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['Week_Number'] = df['Date'].dt.year.astype(str) + '-' + (df['Date'].dt.week + 100).astype(str).str[1:3]


,Date,Distancia,Duration,Average Pace,Average Speed (km/h),Calories Burned,Climb (m),Week_Number,dia_semana
0,2022-02-13 09:53:12,5.01,34:22,6:51,875.0,3220,147,2022-06,06
1,2022-02-06 09:34:34,2.90,33:30,11:34,519.0,1800,93,2022-05,06
2,2022-02-05 09:14:42,8.00,54:55,6:52,875.0,5260,240,2022-05,05
3,2022-02-02 07:39:37,5.09,35:05,6:53,871.0,3370,172,2022-05,02
4,2022-01-30 10:58:47,5.42,37:12,6:52,874.0,3550,164,2022-04,06
...,...,...,...,...,...,...,...,...,...
390,2015-05-31 08:45:46,17.30,1:55:01,6:39,902.0,13810,266,2015-22,06
391,2015-05-24 10:20:25,13.62,1:35:20,7:00,857.0,10800,208,2015-21,06
392,2015-05-03 09:44:33,10.89,1:20:27,7:23,812.0,8430,187,2015-18,06
393,2015-04-26 10:52:26,11.10,1:12:07,6:30,924.0,8290,165,2015-17,06


In [101]:
df['dia_semana'].unique()

array(['06', '05', '02', '03', '01', '04', '00'], dtype=object)

In [102]:
df['vol_semanal'] = df['Distancia'].groupby(df['Week_Number']).transform('sum')

In [103]:
df

,Date,Distancia,Duration,Average Pace,Average Speed (km/h),Calories Burned,Climb (m),Week_Number,dia_semana,vol_semanal
0,2022-02-13 09:53:12,5.01,34:22,6:51,875.0,3220,147,2022-06,06,5.01
1,2022-02-06 09:34:34,2.90,33:30,11:34,519.0,1800,93,2022-05,06,15.99
2,2022-02-05 09:14:42,8.00,54:55,6:52,875.0,5260,240,2022-05,05,15.99
3,2022-02-02 07:39:37,5.09,35:05,6:53,871.0,3370,172,2022-05,02,15.99
4,2022-01-30 10:58:47,5.42,37:12,6:52,874.0,3550,164,2022-04,06,17.25
...,...,...,...,...,...,...,...,...,...,...
390,2015-05-31 08:45:46,17.30,1:55:01,6:39,902.0,13810,266,2015-22,06,17.30
391,2015-05-24 10:20:25,13.62,1:35:20,7:00,857.0,10800,208,2015-21,06,13.62
392,2015-05-03 09:44:33,10.89,1:20:27,7:23,812.0,8430,187,2015-18,06,10.89
393,2015-04-26 10:52:26,11.10,1:12:07,6:30,924.0,8290,165,2015-17,06,16.48


In [104]:
df['vol_semanal_ant'] = df.vol_semanal.shift(-1)

In [105]:
df

,Date,Distancia,Duration,Average Pace,Average Speed (km/h),Calories Burned,Climb (m),Week_Number,dia_semana,vol_semanal,vol_semanal_ant
0,2022-02-13 09:53:12,5.01,34:22,6:51,875.0,3220,147,2022-06,06,5.01,15.99
1,2022-02-06 09:34:34,2.90,33:30,11:34,519.0,1800,93,2022-05,06,15.99,15.99
2,2022-02-05 09:14:42,8.00,54:55,6:52,875.0,5260,240,2022-05,05,15.99,15.99
3,2022-02-02 07:39:37,5.09,35:05,6:53,871.0,3370,172,2022-05,02,15.99,17.25
4,2022-01-30 10:58:47,5.42,37:12,6:52,874.0,3550,164,2022-04,06,17.25,17.25
...,...,...,...,...,...,...,...,...,...,...,...
390,2015-05-31 08:45:46,17.30,1:55:01,6:39,902.0,13810,266,2015-22,06,17.30,13.62
391,2015-05-24 10:20:25,13.62,1:35:20,7:00,857.0,10800,208,2015-21,06,13.62,10.89
392,2015-05-03 09:44:33,10.89,1:20:27,7:23,812.0,8430,187,2015-18,06,10.89,16.48
393,2015-04-26 10:52:26,11.10,1:12:07,6:30,924.0,8290,165,2015-17,06,16.48,16.48


In [106]:
X_train, X_test, y_train, y_test = model_selection.train_test_split( df[['dia_semana','vol_semanal_ant']],
                                                                     df['Distancia'],
                                                                     random_state=1992,
                                                                     test_size=0.25)

In [110]:
onehot = ce.OneHotEncoder(cols=['dia_semana'], use_cat_names=True)


model_xgb = xgb.XGBRegressor(random_state=1992)

full_pipeline_xgb = Pipeline( steps=[("onehot", onehot),
                                     ('modelo', model_xgb) ] )

param_grid = { "modelo__n_estimators":[2,5,10,20],
               "modelo__max_depth":[3,4,5,10],
               "modelo__eta":[0.01, 0.1, 0.2, 0.3, 0.5],
               "modelo__subsample":[0.1, 0.2, 0.25, 0.5, 0.76, 0.8, 0.9]}

search_xgb = model_selection.GridSearchCV(full_pipeline_xgb,
                                          param_grid,
                                          cv=5,
                                          n_jobs=-1,
                                          scoring='neg_root_mean_squared_error') # Declaração

search_xgb.fit(X_train, y_train) # Executa o treinamento!!

best_model_xgb = search_xgb.best_estimator_

In [111]:
best_model_xgb

Pipeline(steps=[('onehot',
                 OneHotEncoder(cols=['dia_semana'], use_cat_names=True)),
                ('modelo',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, enable_categorical=False,
                              eta=0.5, gamma=0, gpu_id=-1, importance_type=None,
                              interaction_constraints='', learning_rate=0.5,
                              max_delta_step=0, max_depth=3, min_child_weight=1,
                              missing=nan, monotone_constraints='()',
                              n_estimators=20, n_jobs=8, num_parallel_tree=1,
                              predictor='auto', random_state=1992, reg_alpha=0,
                              reg_lambda=1, scale_pos_weight=1, subsample=0.76,
                              tree_method='exact', validate_parameters=1,
                              verbosity=

In [112]:
y_test_pred_xgb = best_model_xgb.predict(X_test)
y_test_pred_xgb

array([18.882742 ,  4.5551043, 19.022846 ,  5.2055755,  2.7801988,
       11.275722 , 10.717148 ,  4.5551043,  7.8088913,  6.305791 ,
        9.8771925,  6.6558523,  4.670163 ,  4.670163 ,  5.194719 ,
        9.8178835,  6.100325 ,  6.305791 ,  5.646361 ,  7.413241 ,
        7.724669 ,  6.7616816,  9.8178835,  5.781027 ,  7.5269213,
        6.5007615,  6.406427 ,  7.0415297,  9.4801035,  5.9338827,
        9.8577385,  7.6742253,  6.300694 , 19.036644 ,  9.8178835,
        6.0511713,  3.958691 ,  4.808654 ,  3.958691 ,  7.8088913,
        6.784243 ,  5.979825 ,  6.1309133,  8.301613 ,  8.714183 ,
        7.8088913,  6.7616816,  8.2747555, 16.557001 ,  7.8088913,
        7.018785 , 19.785612 ,  6.459148 ,  7.6667905,  9.4801035,
       20.454626 ,  7.066261 ,  6.7126975,  6.462853 ,  6.0659313,
        7.4310665,  4.670163 ,  5.646361 ,  7.314072 ,  6.0737324,
        9.823291 ,  7.5269213,  2.7801988,  6.0737324,  6.9325256,
        7.9552145,  7.058426 ,  7.3084483, 11.290832 , 10.7983

In [113]:
root_mean_squared_erro_xgb = metrics.mean_squared_error( y_test, y_test_pred_xgb) ** (1/2)

print( "Raiz do Erro Quadrático Médio XGBoost:", (-1) * root_mean_squared_erro_xgb)

Raiz do Erro Quadrático Médio XGBoost: -3.9630448061341745


In [114]:
X_test


,dia_semana,vol_semanal_ant
177,05,36.31
11,03,15.71
131,05,28.15
318,00,15.24
205,03,38.20
...,...,...
383,05,15.89
380,06,6.16
105,06,37.59
287,06,27.52


In [117]:
data = {'dia_semana': '06',
        'vol_semanal_ant': [30]
        }
best_model_xgb.predict(pd.DataFrame(data))

array([10.457467], dtype=float32)